In [1]:
import random
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
PATH = './input/'
df_origin = pd.read_csv(PATH + "webtoon_total_final.csv")

In [3]:
df = df_origin[['title','score', 'genre']]
df.genre = df.genre.str.strip('['']')

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [4]:
tmp = pd.get_dummies(df.genre)

In [5]:
def col_change(df):
    for col in df.columns:
        if ',' in col:
            col_sep = col.split(', ')
            df[col_sep[0]] = df[col_sep[0]] + df[col]
            df[col_sep[1]] = df[col_sep[1]] + df[col]
            df.drop(columns=[f'{col}'], inplace=True)
    return df

In [6]:
col_change(tmp)
tmp

,'드라마','로맨스','무협','스릴러','액션','일상','코믹','판타지','학원'
0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
3228,0,1,0,0,0,0,0,0,0
3229,0,0,0,1,0,0,0,0,0
3230,0,1,0,0,0,0,0,0,0
3231,0,0,0,0,0,0,0,1,1


In [7]:
genre_df = col_change(pd.get_dummies(df.genre))
df = pd.concat([df.title, df.score, genre_df], axis=1)
df

,title,score,'드라마','로맨스','무협','스릴러','액션','일상','코믹','판타지','학원'
0,참교육,9.9510,0,0,0,0,1,0,0,0,0
1,신의 탑,7.3420,0,0,0,0,0,0,0,1,0
2,뷰티풀 군바리,9.7290,1,0,0,0,0,0,0,0,0
3,윈드브레이커,9.5430,0,0,0,0,0,0,0,0,1
4,퀘스트지상주의,9.8890,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3228,진심이 닿다,9.7825,0,1,0,0,0,0,0,0,0
3229,드림사이드,9.8300,0,0,0,1,0,0,0,0,0
3230,김 비서가 왜 그럴까,9.9200,0,1,0,0,0,0,0,0,0
3231,더스크 하울러,9.9350,0,0,0,0,0,0,0,1,1


In [8]:
df.columns = df.columns.str.strip('\'')
df

,title,score,드라마,로맨스,무협,스릴러,액션,일상,코믹,판타지,학원
0,참교육,9.9510,0,0,0,0,1,0,0,0,0
1,신의 탑,7.3420,0,0,0,0,0,0,0,1,0
2,뷰티풀 군바리,9.7290,1,0,0,0,0,0,0,0,0
3,윈드브레이커,9.5430,0,0,0,0,0,0,0,0,1
4,퀘스트지상주의,9.8890,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3228,진심이 닿다,9.7825,0,1,0,0,0,0,0,0,0
3229,드림사이드,9.8300,0,0,0,1,0,0,0,0,0
3230,김 비서가 왜 그럴까,9.9200,0,1,0,0,0,0,0,0,0
3231,더스크 하울러,9.9350,0,0,0,0,0,0,0,1,1


In [9]:
def one_genre(title):
    """
    장르 하나의 
    """
    g_row = df[df.title == title]
    genres = g_row[df.columns[2:]]
    return genres.values

def genres(title_list):
    genre_list = [0]*9
    for title in title_list:
        genre_list = genre_list + one_genre(title)
    return genre_list


## 시청 목록과 장르 유사도 높은 웹툰 중 평점 높은 10개

In [15]:
score = genre_df.to_numpy()

def genre_model(title_list):
    local_score = np.append(score, genres(title_list), axis=0)
    cosine_similar = cosine_similarity(local_score, local_score)
    cosine_similar_data = pd.DataFrame(cosine_similar)

    genre_user = cosine_similar_data.tail(1).T.sort_values(by=cosine_similar_data.columns[-1], ascending=False)
    max_score = genre_user.values[1][0]
    
    max_index = list(genre_user[(genre_user.values) == max_score].index)

    if df.shape[0] in max_index:
        max_index.remove(df.shape[0])

    sorted_df = df.loc[max_index].sort_values(by='score', ascending=False)

    if len(max_index) < 10:
        indlist = sorted_df.index
        return df_origin.loc[indlist]
    
    else:
        min_score = sorted_df.iloc[9].score
        ind = 0
        for i in range(10):
            if sorted_df.iloc[i].score == min_score:
                ind = i
                break
        randnum = sorted_df[sorted_df.score == min_score].shape[0]
        randlist = random.sample(range(ind, ind + randnum), 10 - ind)
        tdf1, tdf2 = sorted_df[:ind], sorted_df.iloc[randlist]
        sorted_df = pd.concat([tdf1, tdf2])
        indlist = sorted_df.index

        return df_origin.loc[indlist]

In [17]:
genre_model(['바니와 오빠들', '신락도', '신의 탑', '오무라이스 잼잼'])

,title,artist,genre,story,image,from,score
2229,창백한 말,이윤희/추혜연,"['로맨스', '판타지']","1835년 프랑스, 마녀로 불리던 소녀의 핏빛 운명",['https://kr-a.kakaopagecdn.com/P/C/783/sharin...,kakao,9.9900
1931,샬롯에게는 다섯 명의 제자가 있다,용용,"['로맨스', '판타지']",세상을 구한 위대한 대마법사 〈샬롯 엘레노어〉 생애 마지막 마법을 시전하고 눈을 감...,['https://kr-a.kakaopagecdn.com/P/C/1796/shari...,kakao,9.9870
2925,남편을 내 편으로 만드는 방법,시루,"['로맨스', '판타지']","판타지 소설 속 남편의 손에 죽는 악역 조연, ‘루드베키아’가 되었다. 이 세계의...",['https://kr-a.kakaopagecdn.com/P/C/2954/shari...,kakao,9.9860
1970,퀴퀴한 일기,2B,"['코믹', '일상']","내 주변에 반드시 한 명은 있을 것만 같은, 어쩌면 나 같기도 한 그녀의 일기. ...",['https://kr-a.kakaopagecdn.com/P/C/1295/shari...,kakao,9.9860
2026,슬프게도 이게 내 인생,슬,"['코믹', '일상']","""벌어먹고 살아야 하는 현생은 너무 힘들어!"" 거친 현대사회에서 이리저리 치이며 ...",['https://kr-a.kakaopagecdn.com/P/C/1575/shari...,kakao,9.9840
2772,남아돌아,남지은/캐러멜,"['코믹', '일상']",남아돌아'는 젊은 층에게 많은 인기를 누리고 있는 캐러맬 작가의 데뷔작,['https://kr-a.kakaopagecdn.com/P/C/9/sharing/...,kakao,9.9825
1855,오무라이스 잼잼,조경규,"['코믹', '일상']",'음식'과 '가족'의 이야기를 담은 에세이 만화. 몇 개라도 먹을 수 있을 것 같...,['https://kr-a.kakaopagecdn.com/P/C/198/sharin...,kakao,9.9820
1976,우리 청춘의 사랑은,손달섭,"['코믹', '일상']",우리의 코믹 남매들이 달달한 러브스토리를 장착해서 다시 돌아왔다 여러분의 배꼽과 연...,['https://kr-a.kakaopagecdn.com/P/C/2600/shari...,kakao,9.9820
3227,그녀가 공작저로 가야 했던 사정,고래,"['로맨스', '판타지']","의문의 사망 이후, 소설 속으로 빙의하게 된 재수생 박은하. 하지만 그녀가 빙의한...",['https://kr-a.kakaopagecdn.com/P/C/2314/shari...,kakao,9.9810
2774,놀아죠 패밀리,강풀/남지은,"['코믹', '일상']",네티즌들과 재밌고 감동적인 이야기를 나누고 싶은 남지은 작가의 데뷔작.,['https://kr-a.kakaopagecdn.com/P/C/5/sharing/...,kakao,9.9800
